In [1]:
import wandb
import pandas as pd
import numpy as np

api = wandb.Api(overrides={'base-url': "https://rosewandb.ucsd.edu"})
runs = api.runs("cht028/gpt-score")

In [2]:
def create_summary_table(runs, row_name, multi_column_name, column_name, metric):
    # Initialize a dictionary for storing the lowest metric values
    lowest_metrics = {}
    run_mapping = {}

    # Find the run with the lowest metric for each combination
    for run in runs:
        try:
            name = run.config[row_name]
            dataset = run.config[multi_column_name]
            window_size = run.config[column_name] if column_name else None
            metric_value = float(run.summaryMetrics[metric])
            metric_value = np.round(metric_value, 3)
        except KeyError:
            continue

        key = (name, dataset) if column_name is None else (name, dataset, window_size)

        if key not in lowest_metrics:
            lowest_metrics[key] = metric_value
            run_mapping[key] = run

    # Prepare the data structure for DataFrame creation
    index = sorted(set(key[0] for key in run_mapping.keys()))
    if column_name:
        datasets = sorted(set(key[1] for key in run_mapping.keys()))
        window_sizes = sorted(set(key[2] for key in run_mapping.keys()))
        multi_columns = pd.MultiIndex.from_product([datasets, window_sizes], names=[multi_column_name, column_name])
    else:
        multi_columns = sorted(set(key[1] for key in run_mapping.keys()))

    # Create the DataFrame
    results_df = pd.DataFrame(index=index, columns=multi_columns)

    # Populate the DataFrame
    for key, run in run_mapping.items():
        if column_name:
            results_df.at[key[0], (key[1], key[2])] = run.summaryMetrics[metric]
        else:
            results_df.at[key[0], key[1]] = run.summaryMetrics[metric]

    # Drop rows and columns with all NaN values
    results_df.dropna(axis=0, how='all', inplace=True)
    results_df.dropna(axis=1, how='all', inplace=True)

    return results_df

In [6]:
results_df = create_summary_table(runs, 'model', 'dataset', 'window_size', 'recalls')
# results_df.to_excel('results.xlsx')
results_df

dataset                       climate                                          \
window_size                       1-1       2-2       3-3       4-4       5-5   
input-copy                   0.520789  0.446278  0.392604  0.364884  0.342771   
text2text-finetuned          0.496785  0.454715  0.438016  0.433671  0.412593   
text2text-inContext          0.430766  0.375155  0.372534   0.34338  0.346231   
text2text-zeroshot           0.241094  0.230845   0.22646  0.212138  0.200409   
textTime2text-finetuned      0.485163  0.461638  0.442923   0.42112  0.414033   
textTime2text-inContext      0.420376  0.363722  0.358025  0.341172  0.332662   
textTime2text-zeroshot       0.244972  0.268936  0.226316  0.200592  0.208892   
textTime2textTime-finetuned  0.506604  0.459136  0.441933  0.430185  0.415599   
textTime2textTime-inContext  0.396538    0.3666  0.351714  0.333553  0.322197   
textTime2textTime-zeroshot   0.201231  0.214184  0.180159   0.17368  0.168308   

dataset                                           medical                      \
window_size                       6-6       7-7       1-1       2-2       3-3   
input-copy                   0.326295  0.329332  0.423755  0.413355  0.405165   
text2text-finetuned          0.401892   0.37978  0.447467  0.450298  0.346914   
text2text-inContext          0.328675  0.324993  0.369323  0.363059  0.364708   
text2text-zeroshot           0.209384  0.231377  0.401611  0.357246  0.339648   
textTime2text-finetuned      0.402985  0.397677  0.441179  0.447348  0.460345   
textTime2text-inContext      0.312502  0.316092  0.383112  0.371067  0.368215   
textTime2text-zeroshot       0.221711  0.215191  0.401914  0.367687  0.343986   
textTime2textTime-finetuned   0.40253  0.394442  0.435328  0.459578  0.456347   
textTime2textTime-inContext  0.313935  0.309827  0.374561  0.380715  0.380191   
textTime2textTime-zeroshot    0.15559  0.163853  0.359137  0.341095  0.314327   

dataset                                                              
window_size                       4-4       5-5       6-6       7-7  
input-copy                   0.392797  0.387188  0.378165       NaN  
text2text-finetuned          0.442198  0.456936  0.447637  0.452174  
text2text-inContext          0.370985  0.376064  0.370549  0.371492  
text2text-zeroshot           0.336318   0.32648  0.323206  0.307393  
textTime2text-finetuned      0.453663  0.450973    0.4546  0.456851  
textTime2text-inContext      0.371098  0.383976  0.377119   0.37529  
textTime2text-zeroshot       0.329937  0.319693  0.314466  0.301096  
textTime2textTime-finetuned  0.454598  0.452186   0.45329  0.452649  
textTime2textTime-inContext  0.379199  0.374125  0.377757  0.374524  
textTime2textTime-zeroshot   0.316411  0.333388  0.290017  0.301166

In [25]:
from pydantic import BaseModel, create_model

def create_answer_format(window_size: int):
    # Dynamically generate fields for the class based on the window size
    fields = {}
    for i in range(3, 3 + window_size):
        fields[f'day_{i}_date'] = (str, ...)
        fields[f'day_{i}_weather_forecast'] = (str, ...)
    
    # Create a dynamic model with the generated fields
    return create_model('AnswerFormat', **fields)

# Example Usage
window_size = 3  # Set window size to 3 dynamically
AnswerFormat = create_answer_format(window_size)
print(AnswerFormat.model_json_schema())

# Create an instance with sample data
data = {
    'day_3_date': '2024-09-01',
    'day_3_weather_forecast': 'Sunny',
    'day_4_date': '2024-09-02',
    'day_4_weather_forecast': 'Rainy',
    'day_5_date': '2024-09-03',
    'day_5_weather_forecast': 'Cloudy'
}

# Validate the data using the dynamically generated model
answer = AnswerFormat(**data)
print(answer)


{'properties': {'day_3_date': {'title': 'Day 3 Date', 'type': 'string'}, 'day_3_weather_forecast': {'title': 'Day 3 Weather Forecast', 'type': 'string'}, 'day_4_date': {'title': 'Day 4 Date', 'type': 'string'}, 'day_4_weather_forecast': {'title': 'Day 4 Weather Forecast', 'type': 'string'}, 'day_5_date': {'title': 'Day 5 Date', 'type': 'string'}, 'day_5_weather_forecast': {'title': 'Day 5 Weather Forecast', 'type': 'string'}}, 'required': ['day_3_date', 'day_3_weather_forecast', 'day_4_date', 'day_4_weather_forecast', 'day_5_date', 'day_5_weather_forecast'], 'title': 'AnswerFormat', 'type': 'object'}
day_3_date='2024-09-01' day_3_weather_forecast='Sunny' day_4_date='2024-09-02' day_4_weather_forecast='Rainy' day_5_date='2024-09-03' day_5_weather_forecast='Cloudy'


In [2]:
import json

# Input dictionary (example with unsorted keys)
data = {
    "day_4_temp": 25.4,
    "day_3_temp": 22.4,
    "day_2_temp": 30,
    "day_5_temp": 28.1
}

# Sort the dictionary by keys
sorted_data = dict(sorted(data.items()))

# Convert the result into a JSON formatted string for better readability
sorted_data_json = json.dumps(sorted_data, indent=4)

# Output the sorted dictionary
print(sorted_data_json)


{
    "day_2_temp": 30,
    "day_3_temp": 22.4,
    "day_4_temp": 25.4,
    "day_5_temp": 28.1
}
